<a href="https://colab.research.google.com/github/vishalmysore/AI/blob/main/CookGPT_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
! pip install transformers

In [2]:
model_name = "NumbersStation/nsql-6B"

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/8.39G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [4]:
table_schema = """CREATE TABLE Recipe (
    recipe_id number,
    location text,
    name text,
    cook_time number,
    total_time number,
    calories number,
    people number,
    is_vegetarian others,
    is_vegan others,
    is_gluten_free others,
    is_dairy_free others,
    is_low_sugar others,
    is_very_healthy others,
    is_cheap others,
    is_very_popular others,
    is_sustainable others,
    instructions text,
    ingredients text,
    nutrients text,
    diet_labels text,
    health_labels text,
    cautions text,
    cuisine_type text,
    meal_type text,
    dish_type text,
    dish_type_others text
    )

CREATE TABLE ratings (
    recipe_id number,
    rating number,
    user_id number,
    date text,
    comment text,
    useful number,
    funny number,
    cool number
)

CREATE TABLE recipe_in_category (
    recipe_id number,
    category_id number
)

CREATE TABLE category (
    category_id number,
    category_name text
)

CREATE TABLE user (
    user_id number,
    user_name text
)

CREATE TABLE recipe_in_cuisine (
    recipe_id number,
    cuisine_id number
)

CREATE TABLE cuisine (
    cuisine_id number,
    cuisine_name text
)
"""



In [5]:
question = "give me recipe from south india ,I need to cook for 2 people and i can cook under 10 minutes? It should be 5 star rated recipe"

prompt = f"""{table_schema}

-- Using valid SQLite, answer the following questions for the tables provided above.

-- {question}

SELECT"""

In [ ]:
question = "give me recipe details with 5 star rating"

prompt = f"""{table_schema}

-- Using valid SQLite, answer the following questions for the tables provided above.

-- {question}

SELECT"""

In [13]:
question = "give all the recipes ingredients for which user vishal has submitted ratings"

prompt = f"""{table_schema}

-- Using valid SQLite, answer the following questions for the tables provided above.

-- {question}

SELECT"""

In [14]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
generated_ids = model.generate(input_ids, max_length=500)
output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
output = 'SELECT' + output.split('SELECT')[-1]

print(output)

SELECT DISTINCT T2.ingredients FROM ratings AS T1 JOIN Recipe AS T2 ON T1.recipe_id = T2.recipe_id JOIN user AS T3 ON T1.user_id = T3.user_id WHERE T3.user_name = "Vishal";


In [15]:
!wget https://tdmdal.github.io/mma-sql-2021/data/northwind.sqlite3

--2024-01-10 23:28:30--  https://tdmdal.github.io/mma-sql-2021/data/northwind.sqlite3
Resolving tdmdal.github.io (tdmdal.github.io)... 185.199.108.153, 185.199.111.153, 185.199.110.153, ...
Connecting to tdmdal.github.io (tdmdal.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 804864 (786K) [application/octet-stream]
Saving to: ‘northwind.sqlite3’

northwind.sqlite3   100%[===================>] 786.00K  --.-KB/s    in 0.007s  

2024-01-10 23:28:30 (105 MB/s) - ‘northwind.sqlite3’ saved [804864/804864]



In [16]:
# load the SQL magic extension
# https://github.com/catherinedevlin/ipython-sql
# this extension allows us to connect to DBs and issue SQL command
%load_ext sql

# now we can use the magic extension to connect to our SQLite DB
# use %sql to write an inline SQL command
# use %%sql to write SQL commands in a cell
%sql sqlite:///northwind.sqlite3

In [17]:
%%sql
SELECT sqlite_version();

 * sqlite:///northwind.sqlite3
Done.


sqlite_version()
3.37.2


In [18]:
%%sql
SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';

 * sqlite:///northwind.sqlite3
Done.


name
Suppliers
Shippers
Employees
Customers
Orders
Categories
Products
OrderDetails


In [21]:
%%sql
SELECT * FROM sqlite_master

 * sqlite:///northwind.sqlite3
Done.


type,name,tbl_name,rootpage,sql
table,Suppliers,Suppliers,2,"CREATE TABLE [Suppliers] ( [SupplierID] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [CompanyName] nvarchar(40) NOT NULL COLLATE NOCASE, [ContactName] nvarchar(30) NULL COLLATE NOCASE, [ContactTitle] nvarchar(30) NULL COLLATE NOCASE, [Address] nvarchar(60) NULL COLLATE NOCASE, [City] nvarchar(15) NULL COLLATE NOCASE, [Region] nvarchar(15) NULL COLLATE NOCASE, [PostalCode] nvarchar(10) NULL COLLATE NOCASE, [Country] nvarchar(15) NULL COLLATE NOCASE, [Phone] nvarchar(24) NULL COLLATE NOCASE, [Fax] nvarchar(24) NULL COLLATE NOCASE, [HomePage] ntext NULL)"
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,Shippers,Shippers,4,"CREATE TABLE [Shippers] ( [ShipperID] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [CompanyName] nvarchar(40) NOT NULL COLLATE NOCASE, [Phone] nvarchar(24) NULL COLLATE NOCASE)"
table,Employees,Employees,5,"CREATE TABLE [Employees] ( [EmployeeID] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [LastName] nvarchar(20) NOT NULL COLLATE NOCASE, [FirstName] nvarchar(10) NOT NULL COLLATE NOCASE, [Title] nvarchar(30) NULL COLLATE NOCASE, [TitleOfCourtesy] nvarchar(25) NULL COLLATE NOCASE, [BirthDate] datetime NULL, [HireDate] datetime NULL, [Address] nvarchar(60) NULL COLLATE NOCASE, [City] nvarchar(15) NULL COLLATE NOCASE, [Region] nvarchar(15) NULL COLLATE NOCASE, [PostalCode] nvarchar(10) NULL COLLATE NOCASE, [Country] nvarchar(15) NULL COLLATE NOCASE, [HomePhone] nvarchar(24) NULL COLLATE NOCASE, [Extension] nvarchar(4) NULL COLLATE NOCASE, [Photo] image NULL, [Notes] ntext NULL, [ReportsTo] int NULL, [PhotoPath] nvarchar(255) NULL COLLATE NOCASE, CONSTRAINT [FK_Employees_Employees] FOREIGN KEY ([ReportsTo]) REFERENCES [Employees] ([EmployeeID]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,Customers,Customers,8,"CREATE TABLE [Customers] ( [CustomerID] nchar(5) NOT NULL, [CompanyName] nvarchar(40) NOT NULL COLLATE NOCASE, [ContactName] nvarchar(30) NULL COLLATE NOCASE, [ContactTitle] nvarchar(30) NULL COLLATE NOCASE, [Address] nvarchar(60) NULL COLLATE NOCASE, [City] nvarchar(15) NULL COLLATE NOCASE, [Region] nvarchar(15) NULL COLLATE NOCASE, [PostalCode] nvarchar(10) NULL COLLATE NOCASE, [Country] nvarchar(15) NULL COLLATE NOCASE, [Phone] nvarchar(24) NULL COLLATE NOCASE, [Fax] nvarchar(24) NULL COLLATE NOCASE, CONSTRAINT [PK_Customers] PRIMARY KEY ([CustomerID]))"
index,sqlite_autoindex_Customers_1,Customers,9,None
table,Orders,Orders,11,"CREATE TABLE [Orders] ( [OrderID] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [CustomerID] nchar(5) NOT NULL COLLATE NOCASE, [EmployeeID] int NULL, [OrderDate] datetime NULL, [RequiredDate] datetime NULL, [ShippedDate] datetime NULL, [ShipVia] int NULL, [Freight] money DEFAULT 0 NULL, [ShipName] nvarchar(40) NULL COLLATE NOCASE, [ShipAddress] nvarchar(60) NULL COLLATE NOCASE, [ShipCity] nvarchar(15) NULL COLLATE NOCASE, [ShipRegion] nvarchar(15) NULL COLLATE NOCASE, [ShipPostalCode] nvarchar(10) NULL COLLATE NOCASE, [ShipCountry] nvarchar(15) NULL COLLATE NOCASE, CONSTRAINT [FK_Orders_Customers] FOREIGN KEY ([CustomerID]) REFERENCES [Customers] ([CustomerID]) ON DELETE NO ACTION ON UPDATE NO ACTION, CONSTRAINT [FK_Orders_Employees] FOREIGN KEY ([EmployeeID]) REFERENCES [Employees] ([EmployeeID]) ON DELETE NO ACTION ON UPDATE NO ACTION, CONSTRAINT [FK_Orders_Shippers] FOREIGN KEY ([ShipVia]) REFERENCES [Shippers] ([ShipperID]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,Categories,Categories,13,"CREATE TABLE [Categories] ( [CategoryID] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [CategoryName] nvarchar(15) NOT NULL COLLATE NOCASE, [Description] ntext NULL, [Picture] image NULL)"
table,Products,Products,14,"CREATE TABLE [Products] ( [ProductID] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [ProductName] nvarchar(40) NOT NULL COLLATE NOCASE, [SupplierID] int NULL, [CategoryID] int NULL, [QuantityPerUnit] nvarchar(20) NULL COLLATE NOCASE, [UnitPrice] money DEFAULT 0 NULL, [UnitsInStoc

In [19]:
%%sql
-- find out relationships between tables using SQL command
SELECT sql
FROM sqlite_master
WHERE name = "Orders"

 * sqlite:///northwind.sqlite3
Done.


sql
"CREATE TABLE [Orders] ( [OrderID] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [CustomerID] nchar(5) NOT NULL COLLATE NOCASE, [EmployeeID] int NULL, [OrderDate] datetime NULL, [RequiredDate] datetime NULL, [ShippedDate] datetime NULL, [ShipVia] int NULL, [Freight] money DEFAULT 0 NULL, [ShipName] nvarchar(40) NULL COLLATE NOCASE, [ShipAddress] nvarchar(60) NULL COLLATE NOCASE, [ShipCity] nvarchar(15) NULL COLLATE NOCASE, [ShipRegion] nvarchar(15) NULL COLLATE NOCASE, [ShipPostalCode] nvarchar(10) NULL COLLATE NOCASE, [ShipCountry] nvarchar(15) NULL COLLATE NOCASE, CONSTRAINT [FK_Orders_Customers] FOREIGN KEY ([CustomerID]) REFERENCES [Customers] ([CustomerID]) ON DELETE NO ACTION ON UPDATE NO ACTION, CONSTRAINT [FK_Orders_Employees] FOREIGN KEY ([EmployeeID]) REFERENCES [Employees] ([EmployeeID]) ON DELETE NO ACTION ON UPDATE NO ACTION, CONSTRAINT [FK_Orders_Shippers] FOREIGN KEY ([ShipVia]) REFERENCES [Shippers] ([ShipperID]) ON DELETE NO ACTION ON UPDATE NO ACTION)"


In [20]:
%%sql
-- take a look at the Employees table first
-- use LIMIT to limit the rows being displayed
SELECT FirstName, LastName, HireDate, Title, Country
FROM Employees
LIMIT 10

 * sqlite:///northwind.sqlite3
Done.


FirstName,LastName,HireDate,Title,Country
Nancy,Davolio,2010-05-01 00:00:00.000,Sales Representative,USA
Andrew,Fuller,2010-08-14 00:00:00.000,"Vice President, Sales",USA
Janet,Leverling,2010-04-01 00:00:00.000,Sales Representative,USA
Margaret,Peacock,2011-05-03 00:00:00.000,Sales Representative,USA
Steven,Buchanan,2011-10-17 00:00:00.000,Sales Manager,UK
Michael,Suyama,2011-10-17 00:00:00.000,Sales Representative,UK
Robert,King,2012-01-02 00:00:00.000,Sales Representative,UK
Laura,Callahan,2012-03-05 00:00:00.000,Inside Sales Coordinator,USA
Anne,Dodsworth,2012-11-15 00:00:00.000,Sales Representative,UK


In [6]:
DATABASE = "northwind.sqlite3"
TABLES = []

In [27]:
!git clone https://github.com/NumbersStationAI/NSQL.git

Cloning into 'NSQL'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 38 (delta 9), reused 7 (delta 7), pack-reused 13
Receiving objects: 100% (38/38), 37.94 KiB | 18.97 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [ ]:
!cd NSQL && pip install -r requirements.txt

In [7]:
import sys
sys.path.append("NSQL/examples/")
from db_connectors import SQLiteConnector
from prompt_formatters import RajkumarFormatter

# Get the connector and formatter
sqlite_connector = SQLiteConnector(
    database_path=DATABASE
)
sqlite_connector.connect()
tab = []
tab.extend(sqlite_connector.get_tables())
print(f"Loading tables: {tab}")

db_schema = [sqlite_connector.get_schema(table) for table in TABLES]
formatter = RajkumarFormatter(db_schema)

Loading tables: ['Categories', 'Customers', 'Employees', 'OrderDetails', 'Orders', 'Products', 'Shippers', 'Suppliers', 'sqlite_sequence']
